# <center>**MIMO NOMA with two Clusters**<center>

In this notebook, we will try to summarize the system model as well as the code used for the simulation part. We will provide comments and explanation for both parts to make it easy to reproduce the result or detect any bugs/errors in the code. The current work is part of a collobaration between CTTC and KAUST.

## **1. System Model**

The received signal is given by

$$ \bf{y = H x + n = H P s + n} $$


where $\textbf{H} \in \mathbb{C}^{N \times K}$, $\textbf{P} \in \mathbb{C}^{K \times K}$, $ \textbf{x} \in \mathbb{C}^{K \times 1}$, $\textbf{n} \in \mathbb{C}^{N \times 1}$. Here, $N$ denotes the number of antennas at the base station (BS), and $K$ is the number of users.


The matrix $\textbf{H} = [h_1, h_2, \dots, h_K]$ where $h_k \sim CN(0,\textbf{I}_N) $. We can later on see how things will work if we consider the correlated case.

The matrix $\textbf{P}$ is defined as $\textbf{P} = \text{diag}(P_k)$, where $P_k = d_k^{-\beta} 10^{\frac{\alpha_k}{10}}$ such that
- $d_k^{-\beta}$ is the path-loss where $d_k$ is the distance from the BS to the user $k$, and $\beta$ is the path-loss exponent,
- the lognormal fading exponent $\alpha_k \sim N(0,\sigma_{\alpha}^2)$.


The users are randomly distributed within a rectangle with dimension $(X_0,Y_0)$ from the BS, i.e., the distance $d_k = \sqrt{X_k^2 + Y_k^2}$, where $X_k$ is a uniform RV with range $[0,X_0]$, and $Y_k$ is a uniform RV with range $[0,Y_0]$. This can be later on changed if the model of the distribution of the users is very simple.



Based on Xavi's notes, the output SNR of a user $i$ in the first cluster is given by

$$ SNR_i^{(1)} = P_i \bf{h_i^H} \left(H_{(i)} P_{(i)} H_{(i)}^H + \sigma^2 I \right)^{-1} h_i$$

If we assume that we divide the users into two clusters, $\mathcal{I}_1$ and $\mathcal{I}_1$, the sum rate associated with the first stage is given by

$$ R_1 = \sum_{k \in \mathcal{I}_1}{\log\left(1 + SNR_k^{(1)}\right)}$$

The rate of the second stage is given by

$$ R_2 = \sum_{k \in \mathcal{I}_2}{\log\left(1 + SNR_k^{(2)}\right)},$$

where the SNR of user $i$ in the second cluster is given by

$$ SNR_i^{(2)} = P_i \bf{h_i^H} \left(H_{(\mathcal{I}_1 \cup \{i\})} P_{(\mathcal{I}_1 \cup \{i\})} H_{(\mathcal{I}_1 \cup \{i\})}^H + \sigma^2 I \right)^{-1} h_i$$

The best clustering strategy will be base on maximizing the sum rate $R_1 + R_2$, other possible criteria are maximizing the minimum between $R_1$ and $R_2$ or the product $R_1 \times R_2$. First, we will focus on the first criterion, and then we could consider the other ones later on.

## **2. Dataset Construction**

We will start by importing some of the pakages that we will used later on in the simulation part

In [1]:
from itertools import product, combinations
import numpy as np
from numpy.linalg import inv
import h5py
import pandas as pd
from PIL import Image
from numpy import linalg as LA
import os
from sklearn.model_selection import train_test_split
from PIL import Image
from matplotlib.pyplot import imshow
import seaborn as sns
from sklearn import metrics
from IPython.display import clear_output
%matplotlib inline

### 2.1 System Model Parameters

In order to be able to run some numerical simulations, construct the neural network, and tune its parameters, we need to sepecify the parameters that characterize the environment of interest, and start constructing the dataset that will be used to train the neural network.

In [2]:
beta = 3.76 # 3GPP propogation environment
sigma = 1 # variance of the noise
sigma_alpha_db = 7 # variance of $\alpha$ in dB
sigma_alpha = 10**(sigma_alpha_db/10)
N = 4 # number of antennas in the BS
K = 10 # number of users (K >= N)
X0 = 0.500 # parameter of the uniform distribution of the user on the x-axis
Y0 = 0.500 # parameter of the uniform distribution of the user on the x-axis
nsize = 10000 # size of the dataset

## 2.2 Helper Functions

The function `clusters`. This function takes in a set A and gives all possible (two) clusters (without repetition) from a set A except the set A and empty set. 

In [3]:
def clusters(A):
    subsets = []
    for i in range(1, len(A)):
        subsets.append(list(combinations(A,i)))
    combos = []
    for i in range(1, int(1+len(subsets)/2)):
        combos.extend(list(product(subsets[i-1], subsets[-i])))
    if not len(A) % 2:
        combos.extend(list(combinations(subsets[int(len(A)/2)-1], 2)))
    return [combo for combo in combos if not set(combo[0]) & set(combo[1])]

Let's test this function on an example to see how it works. Let's consider a set $A = \{1, 2, 3\}$.

In [4]:
A = {1, 2, 3, 4}
print(clusters(A))

[((1,), (2, 3, 4)), ((2,), (1, 3, 4)), ((3,), (1, 2, 4)), ((4,), (1, 2, 3)), ((1, 2), (3, 4)), ((1, 3), (2, 4)), ((1, 4), (2, 3))]


As you can see, this give us all possible clusters from $A$ without repetition and excluding the cluster $(A, \emptyset)$. Later on, since the order of the cluster matters, we will need to take into consideration the other possibilities where we need to just flip the elements of each tuple and also add the trivial cluster $(A, \emptyset)$.

Now, we will define a function that generates a sample channel matrix $\textbf{H}$ given the parameters $N$ and $K$.

In [5]:
def channel_matrix(N, K):
    
    d = 0.5
    AoA=np.random.uniform(-0.5*np.pi,0.5*np.pi,size=(1,K))      
    # Uncomment this if you want AoAs sorted in asccending order when generated
    #AoA=np.sort(AoA)   
    AoA_deg=180.0/np.pi*AoA    
    phase_shifts_gen=2*np.pi*d*np.sin(AoA)    
    aux=np.reshape(np.arange(0,N),(N,1))    
    phase_shifts=np.dot(aux,phase_shifts_gen)   
    amplitudes=np.ones((1,K))    
    amplitudes_extd = np.dot(np.ones((N,1)), amplitudes)   
    H=np.multiply(amplitudes_extd, np.exp(-1j*phase_shifts)) #Element by element product.
    
    return H

Let's test this function! 

In [6]:
H = channel_matrix(N, K)
H

array([[ 1.        +0.j        ,  1.        +0.j        ,
         1.        +0.j        ,  1.        +0.j        ,
         1.        +0.j        ,  1.        +0.j        ,
         1.        +0.j        ,  1.        +0.j        ,
         1.        +0.j        ,  1.        +0.j        ],
       [-0.98175724-0.19013868j,  0.9029751 +0.42969289j,
        -0.20593392+0.9785659j , -0.81386709-0.58105109j,
         0.25031341+0.96816486j, -0.81126329+0.58468101j,
         0.99976682+0.02159391j, -0.96149039-0.27483854j,
         0.54716974+0.83702167j,  0.85826555+0.51320584j],
       [ 0.92769457+0.37334005j,  0.63072805+0.77600395j,
        -0.91518244-0.40303982j,  0.32475927+0.94579671j,
        -0.87468639+0.4846893j ,  0.31629624-0.94866047j,
         0.99906741+0.04317775j,  0.84892755+0.52850924j,
        -0.40121055+0.91598586j,  0.47323952+0.8809338j ],
       [-0.83978447-0.54291992j,  0.23608834+0.9717316j ,
         0.58286813-0.81256676j,  0.28524533-0.95845454j,
        -0.

Now, we will define the function `power_matrix` that generates a sample of the diagonal elements of the power matrix $\bf{P}$.

In [7]:
def power_matrix(X0, Y0, K, sigma_alpha):
    alpha = sigma_alpha*np.random.randn(K)
    X = np.random.uniform(low=0.0, high=X0, size=K)
    Y = np.random.uniform(low=0.0, high=Y0, size=K)
    d = np.sqrt(X**2+Y**2)
    P = 10**(alpha/10)*d**(-beta)
    return P

In [8]:
P = power_matrix(X0, Y0, K, sigma_alpha)
P

array([ 82.11804719,  97.90910592,  95.76338197,  32.92304463,
       153.21237372, 397.41479225, 800.23770596,   2.43234593,
        15.45701145,   1.72570267])

Note that the function gives only the diagonal elements of $\bf{P}$, later on, we will build the corresponding matrix from these diagonal elements. Since we will use hermetian matrices, a small function `hermetian` is implemented for this purpose

In [9]:
def hermetian(H):
    return H.conj().T

Now, we will define a function that computes $SNR_i^{(1)}$, the SNR of user $i$ in the first cluster

In [10]:
def SNR1(H, P, sigma, i):
    K = H.shape[1]
    N = H.shape[0]
    P_i= P[i]
    h_i = H[:,i][:, np.newaxis]
    P = np.delete(P, i)
    set_minus_i = list(range(K))
    set_minus_i.pop(i)
    SNR = P_i*hermetian(h_i).dot(inv(H[:,set_minus_i].dot(np.diag(P)).dot(hermetian(H[:,set_minus_i])) + sigma**2*np.identity(N))).dot(h_i)
    return SNR.real

Similarly, using the formula explained above, we implement a function that computes $SNR_i^{(2)}$, the SNR of user $i$ in the second cluster, give the set of user of the first cluster *set_I1*

In [11]:
def SNR2(H, P, sigma, i, set_I1):
    K = H.shape[1]
    N = H.shape[0]
    P_i= P[i]
    set_0 = set_I1 + [i]
    h_i = H[:,i][:, np.newaxis]
    P = np.delete(P, set_0)
    set_minus_i = list(range(K))
    set_minus_i = [j for j in set_minus_i if j not in set_0]
    SNR = P_i*hermetian(h_i).dot(inv(H[:,set_minus_i].dot(np.diag(P)).dot(hermetian(H[:,set_minus_i])) + sigma**2*np.identity(N))).dot(h_i)
    return SNR.real

The function `labelling` is used to determine the best clustering strategy given a certain criteria (e.g., maximum of sum rate). This function uses exhuastive search to determine the clustering strategy and will be used to label the examples in our dataset.

In [12]:
def labelling(N, K, H, P):
    A = set(range(K))
    set_total = clusters(A)
    set_total.append((tuple(A),()))
    set_1 = [list(set_total[i][0]) for i in range(2**(K-1))]
    set_2 = [list(set_total[i][1]) for i in range(2**(K-1))]
    set_I1 = set_1 + set_2
    set_I2 = set_2 + set_1
    R_1 = np.zeros(2**K)
    R_2 = np.zeros(2**K)
    for i in range(2**K): #each possible scenario
        SNR_1 = [SNR1(H, P, sigma, s) for s in set_I1[i]] # SNR_i^(1) for i in I_1
        SNR_2 = [SNR2(H, P, sigma, s, set_I1[i]) for s in set_I2[i]] # SNR_i^(2) for i in I_2
        R_1[i] = np.sum(np.log(1+np.array(SNR_1)))
        R_2[i] = np.sum(np.log(1+np.array(SNR_2)))
    # define the criterion to select the best configuration (max(R1 + R2) or fairness: max(min(R1,R2)) or max R1*R2)
    sum_rate = R_1 + R_2
    best_index = np.argmax(sum_rate)
    return set_I1[best_index], set_I2[best_index]

The function `DataSet` is used to construct the dataset used to train the neural network. We will save, for each scenario, i.e., each sampled $\bf{H}$ and $\bf{P}$, the matrix $\bf{H \times \sqrt{P}}$ in a folder (already created) called `Data` as h5py file. In other words, the folder `Data` will contain *nsize* h5py files once the function finish running. Later on, we will use the real, imaginary and angle parts of each instance as stack them in one matrice to be fed as the input of the neural netwrok. We will get also a dataframe as a csv file containing for each matrix $\bf{H \times \sqrt{P}}$ , the corresponding clustering strategy as a $1 \times K$ vector. 

In [13]:
def DataSet(N, K, X0, Y0, sigma_alpha, nsize):
    # creating empty dataframe with columns
    col_names=['Power' + str(i+1) for i in range(K)] + ['User' + str(i+1) for i in range(K)]
    column_names=['H'] + ['P'] + ['User' + str(i+1) for i in range(K)]
    df = pd.DataFrame(columns = column_names, index = range(nsize))
    df_power = pd.DataFrame(columns = col_names, index = range(nsize))
    for j in range(nsize):
        
        clear_output(wait = True)
        
        H = channel_matrix(N, K)
        P = power_matrix(X0, Y0, K, sigma_alpha)
        
        power = []
        for i in range(K):
            h_i = H[:,i][:, np.newaxis]
            P_i = P[i]
            power.append(LA.norm(h_i)**2*P_i)
        
        # store the matrices for each scenario
        with h5py.File('Data/H_'+ str(j+1) +'.h5', 'w') as hf:
            hf.create_dataset("H_" + str(j+1),  data=H)
        
        with h5py.File('Data/P_'+ str(j+1) +'.h5', 'w') as hf:
            hf.create_dataset("P_" + str(j+1),  data=np.diag(P))
                    
        # get the best clustering strategy
        set0, set1 = labelling(N, K, H, P)
        # transform it to one vector
        labels = np.zeros(K)
        for i in range(K):
            if  i in set0:
                labels[i] = 0
            else:
                labels[i] = 1
        
        df.loc[j] = ['H_'+ str(j+1) +'.h5'] + ['P_'+ str(j+1) +'.h5'] + list(labels)
        df_power.loc[j] = list(power) + list(labels)
        print('Progress:', np.round((j+1)/nsize*100,2), '%')
    
    df.to_csv('data.csv', encoding='utf-8')
    df_power.to_csv('data_power.csv', encoding='utf-8')
    return df, df_power

Let's test this function!

In [ ]:
df = DataSet(N, K, X0, Y0, sigma_alpha, nsize)